# Supervisor Agent Constructuion/Testing 

In [1]:
import sys 
import subprocess

# get root of current repo and add to our path
root_dir = subprocess.check_output(["git", "rev-parse", "--show-toplevel"], stderr=subprocess.DEVNULL).decode("utf-8").strip()

sys.path.append(root_dir)

## S3 Scanning Tool 

In [1]:
import boto3

def get_client_with_role(role_arn: str, aws_service: str, session_name: str, region: str = "us-east-1"):
    # start sts session
    sts = boto3.client("sts")

    # assume role + grab credentials
    creds = sts.assume_role(
        RoleArn=role_arn,
        RoleSessionName=session_name
    )["Credentials"]

    # create a new session with the assumed role credentials
    return boto3.client(
        aws_service,
        aws_access_key_id=creds["AccessKeyId"],
        aws_secret_access_key=creds["SecretAccessKey"],
        aws_session_token=creds["SessionToken"],
        region_name=region
    )

In [4]:
from datetime import datetime

def get_yesterdays_files(s3_client):
    today = datetime.today().strftime("%m%d%Y")
    yesterdays_files = []
    paginator = s3_client.get_paginator("list_objects_v2")
    for page in paginator.paginate(Bucket="agentic-de", Prefix=f"bronze/{today}/"):
        for obj in page.get("Contents", []):
            key = obj["Key"]
            yesterdays_files.append(key)
    return yesterdays_files

In [7]:
def get_file_meta(key, s3_client):
    obj = s3_client.get_object(Bucket="agentic-de", Key=key)
    content = obj["Body"].read(500).decode("utf-8")
    return f"File: {key}\nContent Preview:\n{content}"

In [5]:
import os 

# create s3 client 
s3 = get_client_with_role(
    role_arn=os.getenv("DIGI_INNO_ROLE_ARN"), 
    aws_service="s3",
    session_name="digi-inno-s3"
)

files = get_yesterdays_files(s3)

## Airflow Task Runner

In [5]:
from utils.airflow import AirflowClient
import os 

airflow = AirflowClient(
    username=os.getenv("AIRFLOW_USERNAME"),
    password=os.getenv("AIRFLOW_PASSWORD"),
)

airflow_res = airflow.trigger_dag("transform_pbs_data")